<a href="https://colab.research.google.com/github/z-gard/analysis/blob/main/notebooks/population_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 人口マスタ

In [1]:
!pip install -q geocoder
!pip install -q --upgrade xlrd
!pip install -q japanize-matplotlib

     |████████████████████████████████| 98 kB 3.7 MB/s 
     |████████████████████████████████| 96 kB 2.7 MB/s 
     |████████████████████████████████| 4.1 MB 5.3 MB/s 


In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
DIR_NAME = '/content/drive/MyDrive/z-gard/data'

In [21]:
import os
import requests
import json
import pandas as pd
from urllib.parse import urlparse
import geocoder
import math
import io
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [17]:
def get_data_csv_url(url, encoding='utf-8', excel=False):
    res = requests.get(url)
    if res.status_code == 200:
        if not excel:
            return pd.read_csv(io.BytesIO(res.content), encoding=encoding)
        else:
            return pd.read_excel(io.BytesIO(res.content))
    else:
        raise Exception(f'status_code: {res.status_code} ({url})')

def number_to_kanji(str):
    return str.translate(str.maketrans({'１':'一', '２':'二', '３':'三', '４':'四', '５':'五', '６':'六', '７':'七', '８':'八', '９':'九'}))

def add_pref_district(df_data, district, is_master=False):
    df_data['都道府県'] = '東京都'
    df_data['市区町村'] = district
    if is_master:
        df_data['町丁目'] = df_data['町丁目'].apply(number_to_kanji)
        df_data['町名'] = df_data['町丁目'].str.replace('[一二三四五六七八九十]丁目', '', regex=True)

# 緯度経度取得
def add_lat_lon(df_master):
    lats = []
    lons = []
    for i, row in df_master.iterrows():
        ret = geocoder.osm(row['都道府県'] + row['市区町村'] + row['町丁目'], timeout=5)
        if not ret.ok:
            print(i, row['市区町村'], row['町丁目'])
        lats.append(ret.lat)
        lons.append(ret.lng)
    df_copy = df_master.copy()
    df_copy['緯度'] = lats
    df_copy['経度'] = lons
    return df_copy

### 港区

In [4]:
minato_master_url = 'https://opendata.city.minato.tokyo.jp/dataset/213a32cd-85de-4c30-aaf9-7164c5bf06c9/resource/bc0beb3f-ca1d-45a3-87ea-8e190bb34d27/download/chomokubetsu_2022.csv'
df_minato_master = get_data_csv_url(minato_master_url)
df_minato_master = df_minato_master.dropna(subset=['人口合計［人］'])
add_pref_district(df_minato_master, '港区', True)
df_minato_master = df_minato_master.sort_values('年月日［西暦］', ascending=False).drop_duplicates(subset=['町丁目'])
df_minato_master = df_minato_master.drop(columns=['年月日［西暦］'])
df_minato_master = df_minato_master.sort_values('町丁目').reset_index(drop=True)
df_minato_master.rename(columns={'人口男［人］': '人口男', '人口女［人］': '人口女', '人口合計［人］': '人口合計'}, inplace=True)
df_minato_master.drop(columns=['地区'], inplace=True)
print(df_minato_master.shape)

(117, 8)


In [5]:
df_minato_master.head()

,町丁目,世帯数,人口男,人口女,人口合計,都道府県,市区町村,町名
0,三田一丁目,2251.0,1809.0,2166.0,3975.0,東京都,港区,三田
1,三田三丁目,1086.0,798.0,878.0,1676.0,東京都,港区,三田
2,三田二丁目,2404.0,2022.0,2481.0,4503.0,東京都,港区,三田
3,三田五丁目,3617.0,2930.0,3629.0,6559.0,東京都,港区,三田
4,三田四丁目,2017.0,1606.0,1959.0,3565.0,東京都,港区,三田


### 墨田区

In [10]:
sumida_master_url = 'https://www.city.sumida.lg.jp/kuseijoho/sumida_info/opendata/opendata_ichiran/population_data/setai_all_data.files/setai_all_2208.csv'
df_sumida_master = get_data_csv_url(sumida_master_url, 'shift-jis')
add_pref_district(df_sumida_master, '墨田区', True)
df_sumida_master = df_sumida_master.sort_values('町丁目').reset_index(drop=True)
df_sumida_master.rename(columns={'世帯数[世帯]': '世帯数', '男[人]': '人口男', '女[人]': '人口女', '総数[人]': '人口合計'}, inplace=True)
print(df_sumida_master.shape)

(104, 8)


In [11]:
df_sumida_master.head()

,町丁目,世帯数,人口男,人口女,人口合計,都道府県,市区町村,町名
0,両国一丁目,1206,888,884,1772,東京都,墨田区,両国
1,両国三丁目,1396,1016,1021,2037,東京都,墨田区,両国
2,両国二丁目,1422,1065,1112,2177,東京都,墨田区,両国
3,両国四丁目,1405,1093,1087,2180,東京都,墨田区,両国
4,亀沢一丁目,1290,1031,1099,2130,東京都,墨田区,亀沢


### 練馬区

In [12]:
nerima_master_url = 'https://www.city.nerima.tokyo.jp/kusei/tokei/opendata/opendatasite/tokei_kusei/chouchoubetu.files/202207chouchoubetu.csv'
df_nerima_master = get_data_csv_url(nerima_master_url, 'shift-jis')
add_pref_district(df_nerima_master, '練馬区', True)
df_nerima_master = df_nerima_master[df_nerima_master['町丁目'].str.contains('丁目')]
df_nerima_master.rename(columns={'世帯数[世帯]': '世帯数', '人口（男）[人]': '人口男', '人口（女）[人]': '人口女', '計[人]': '人口合計'}, inplace=True)
print(df_nerima_master.shape)

(198, 8)


In [13]:
df_nerima_master.head()

,町丁目,世帯数,人口男,人口女,人口合計,都道府県,市区町村,町名
2,旭丘一丁目,3350,2478,2392,4870,東京都,練馬区,旭丘
3,旭丘二丁目,1552,1179,1268,2447,東京都,練馬区,旭丘
5,小竹町一丁目,3137,2491,2703,5194,東京都,練馬区,小竹町
6,小竹町二丁目,2596,2034,2315,4349,東京都,練馬区,小竹町
9,羽沢一丁目,927,672,697,1369,東京都,練馬区,羽沢


### 台東区

In [14]:
taito_master_url = 'https://www.city.taito.lg.jp/kusei/shokai/tokei/setai/setaisu/r04chocho.files/Z_cho0408.csv'
df_taito_master = get_data_csv_url(taito_master_url, 'shift-jis')
df_taito_master = df_taito_master[df_taito_master['町名'] != '総数'].rename(columns={'町名': '町丁目'})
add_pref_district(df_taito_master, '台東区', True)
df_taito_master['人口男'] = df_taito_master['人口　男　日本人'].str.replace(',', '').astype(int) + df_taito_master['人口　男　外国人'].str.replace(',', '').astype(int)
df_taito_master['人口女'] = df_taito_master['人口　女　日本人'].str.replace(',', '').astype(int) + df_taito_master['人口　女　外国人'].str.replace(',', '').astype(int)
df_taito_master['世帯数'] = df_taito_master['世帯数　計'].str.replace(',', '').astype(int)
df_taito_master['人口合計'] = df_taito_master['人口　男女計　計'].str.replace(',', '').astype(int)
df_taito_master = df_taito_master[['町丁目', '町名', '都道府県', '市区町村', '世帯数', '人口男', '人口女', '人口合計']]
print(df_taito_master.shape)

(108, 8)


In [15]:
df_taito_master.head()

,町丁目,町名,都道府県,市区町村,世帯数,人口男,人口女,人口合計
1,台東一丁目,台東,東京都,台東区,1478,1291,960,2251
2,台東二丁目,台東,東京都,台東区,1544,1245,1112,2357
3,台東三丁目,台東,東京都,台東区,1887,1495,1441,2936
4,台東四丁目,台東,東京都,台東区,1036,860,765,1625
5,柳橋一丁目,柳橋,東京都,台東区,925,731,652,1383


### 人口マスターデータ

In [16]:
df_tokyo_district_master = pd.concat([
    df_minato_master,
    df_sumida_master,
    df_nerima_master,
    df_taito_master,
]).reset_index(drop=True)
print(df_tokyo_district_master.shape)

(527, 8)


In [18]:
%%time
df_tokyo_district_master = add_lat_lon(df_tokyo_district_master)

218 墨田区 錦糸三丁目
CPU times: user 8.38 s, sys: 590 ms, total: 8.97 s
Wall time: 4min 28s


In [22]:
df_tokyo_district_master.head()

,町丁目,世帯数,人口男,人口女,人口合計,都道府県,市区町村,町名,緯度,経度
0,三田一丁目,2251.0,1809.0,2166.0,3975.0,東京都,港区,三田,35.653750,139.741676
1,三田三丁目,1086.0,798.0,878.0,1676.0,東京都,港区,三田,35.643955,139.743205
2,三田二丁目,2404.0,2022.0,2481.0,4503.0,東京都,港区,三田,35.649845,139.741681
3,三田五丁目,3617.0,2930.0,3629.0,6559.0,東京都,港区,三田,35.646272,139.738259
4,三田四丁目,2017.0,1606.0,1959.0,3565.0,東京都,港区,三田,35.643942,139.739434


In [23]:
df_tokyo_district_master.to_csv(os.path.join(DIR_NAME, 'population_master.csv'), index=False)